 # <strong>Elevation controls on road network robustness in U.S. barrier islands</strong>
 ## <strong>- Download drivable road networks for the Atlantic and Gulf barrier islands -</strong>
 ### The purpose of this notebook is to download drivable road networks from OSM and retrieve the elevation of each network node using CUDEM and the exceedance probability of extreme water events associated to each node elevation.

In [2]:
# Packages

import os
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import rasterio
import numpy as np

In [3]:
### Set working directory
os.chdir('E:\\Networks')

In [ ]:
### Extract road networks from OSM, retrieve node elevation using CUDEM and calculate exceedance probability for each node (in return periods)

# Create folder if it doesn't exist
outdir= '.\\Data\\Roads'
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
    
# Loop through folder containing barriers outlines (original polygons, not buffers) and use them as a mask to extract OSMnx drivable networks

rootdir = '.\\Data\\Barriers\\Barriers_AtlGulf'
extensions = ('.shp')
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier_name= file.replace('.shp', '')
            print(barrier_name)
            
            # try-statment handles exceptions that generate errors or make Python stop when networks cannot be pulled (because the barrier islands have no drivable roads)
            try: 
                # pull drivable networks using cleaned barrier polygons
                poly = gpd.read_file(file_path)
                poly_geo = poly['geometry'].iloc[0]
                poly_geo = poly_geo.buffer(0)
                G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                
                if len(G.nodes(data=True))>100: # keep only networks that have more than 100 nodes
                    
                    # give to each node a new index based on integers from 0 and then add the osmid as an attribute
                    osmids = list(G.nodes)
                    G = nx.relabel.convert_node_labels_to_integers(G)
                    osmid_values = {k:v for k, v in zip(G.nodes, osmids)}
                    nx.set_node_attributes(G, osmid_values, 'initial_osmid')

                    # retrieve nodes and edges in geodataframes
                    nodes, edges = ox.graph_to_gdfs(G)

                    # create an index for nodes in the geodataframe
                    nodes['index'] = range(0, len(nodes))

                    # open corresponding CUDEM mosaic
                    cudem= rasterio.open('.\\Data\\CUDEM\\CUDEM_Mosaic\\{0}.tif'.format(barrier_name))

                    # convert nodes gdf to the cudem crs 
                    nodes = nodes.to_crs("epsg:4269")

                    # for each node in the network, extract elevations using CUDEM raster and add as a new column
                    values=[]
                    for point in nodes['geometry']:
                        x = point.xy[0][0]
                        y = point.xy[1][0]
                        row, col=cudem.index(x,y)
                        value=cudem.read(1)[row,col]
                        values.append(value)   
                    nodes['elevation']=values
                
                    # calculate exceedance probability for each network node according to their elevation 
                    barrier_name= file.replace('_geo.shp', '')
                    exceed = pd.read_csv(".\\Data\\Exceedance\\Probability\\{0}_Exceedance.csv".format(barrier_name), sep=",", header=0)
                    for i in range(0,len(nodes)):
                        node_elev= float(nodes.elevation[i])
                        exceed_x= exceed.MaxWL
                        exceed_y= exceed.Return_Pd
                        exceedance= np.interp(node_elev, exceed_x, exceed_y)
                        nodes.at[i,'exceedance']=exceedance

                    # set elevation and exceedance as attributes in the graph and save graph 
                    nx.set_node_attributes(G, nodes.elevation, 'Elevations')
                    nx.set_node_attributes(G, nodes.exceedance, 'Exceedance')
                    ox.save_graphml(G, filepath='.\\Data\\Roads\\{0}.graphml'.format(barrier_name))
                
            except:
                continue
            

FL10_geo
FL11_geo
FL12_geo
FL13_geo
FL14_geo
FL15_geo
FL16_geo
FL17_geo
FL18_geo
FL19_geo
FL1_geo
FL20_geo
FL21_geo
FL23_geo
FL24_geo
FL25_geo
FL26_geo
FL27_geo
FL28_geo
FL29_geo
FL2_geo
FL30_geo
FL31_geo
FL32_geo
FL33_geo
FL34_geo
FL35_geo
FL36_geo
FL37_geo
FL38_geo
FL39_geo
FL3_geo
FL40_geo
FL4_geo
FL5_geo
FL6_geo
FL7_geo
FL8_geo
FL9_geo
GA10_geo
GA1_geo
GA2_geo
GA3_geo
GA4_geo
GA5_geo
GA6_geo
GA7_geo
GA8_geo
GA9_geo
LA10_geo
LA11_geo
LA12_geo
LA13_geo
LA14_geo
LA15_geo
LA16_geo
LA17_geo
LA18_geo
LA19_geo
LA1_geo
LA20_geo
LA21_geo
LA22_geo
LA2_geo
LA3_geo
LA4_geo
LA5_geo
LA6_geo
LA7_geo
LA8_geo
LA9_geo
MA10_geo
MA11_geo
MA12_geo
MA13_geo
MA14_geo
MA1_geo
MA2_geo
MA3_geo
MA4_geo
MA5_geo
MA6_geo
MA7_geo
MA8_geo
MA9_geo
NC10_geo
NC11_geo
NC12_geo
NC13_geo
NC14_geo
NC15_geo
NC16_geo
